<img src="Maslow's_Hierarchy_of_Needs.svg.png" height = 600px>

In [35]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import urllib.parse # combine URL components into URL string
import wikipediaapi # query wikipedia through api

from pytrends.request import TrendReq # Google Trends API

from statsmodels.tsa.seasonal import STL # seasonal decompositions
import statsmodels.tsa.stattools as smt


import pickle #  to serialize and deserialize objects in Python
from requests.exceptions import RequestException
import requests
from json.decoder import JSONDecodeError
from scipy import signal
import warnings
from urllib import request
from bs4 import BeautifulSoup
import json
import pytrends
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import wikipediaapi
import pprint as pp

## Create a dataframe of pageviews per topic from a .txt using Wikipedia API

In [97]:
def create_dataframe(name_file):
    """
    Creates a dataframe from a text file
    param: name_file: name of the text file
    return: dataframe with the text file
    """
    df = pd.read_csv(name_file, delimiter="\t", header=None, names=['Topics'])
    df.drop_duplicates(inplace=True)
    return df

# Take only starting from the second word in each row
def remove_space(df):
    return df['Topics'].apply(lambda x: x.strip().replace(' ', '_'))

# Parse the topics into the URL format
def parse_topics_into_df(df, start_time, end_time):
    # change the spaces to underscores
    df['url'] = np.zeros(len(df))
    for index, row in df.iterrows():
        topic_value = row['Topics']
        df.loc[index, 'url'] = f'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/{topic_value}/monthly/{start_time}/{end_time}'

    return df

# Create a new dataframe with timestamp from starting date to ending date
def create_dataframe_timestamp(starting_date, ending_date):
    df_timestamp = pd.DataFrame()
    df_timestamp['Timestamp'] = pd.date_range(start=starting_date, end=ending_date, freq='MS')
    return df_timestamp

# Define a function to fetch data from the URL and handle errors
def fetch_and_parse_url(url):
    try:
        request.urlopen(url).read()
        return True
    except request.HTTPError as e:
        if e.code == 404:
            return None  # or any other value or action you prefer for 404 errors
        else:
            return None  # or handle other HTTP errors as needed
    except Exception as e:
        
        return None  # or handle other exceptions as needed

def get_pageviews_wiki(url):
    """
    Gets the weekly pageviews for one Wikipedia page in one language in the desired period
    param: url: url of the Wikipedia page
    param: start_date: beginning of the desired period 
    param: end_date: end of the desired period 
    return: dataframe column with the monthly pageviews
    """
    html = request.urlopen(url).read()
    soup = BeautifulSoup(html,'html.parser')
    site_json=json.loads(soup.text)
    df=pd.DataFrame(site_json['items'])
    df=df['views']
    return df

def scrape_pageviews(df):
    pageview = pd.DataFrame()
    pageview['Timestamp'] = create_dataframe_timestamp('2019-01-01', '2020-07-31')['Timestamp']

    # Loop through the rows of the DataFrame and append the results of the function to the DataFrame
    for index, row in df.iterrows():
        url = row['url']
        if fetch_and_parse_url(url):
            pageview_solo = pd.DataFrame()
            pageview_solo[row['Topics']] = get_pageviews_wiki(url)
            pageview = pd.concat([pageview, pageview_solo], axis=1)

    return pageview


def create_dataframe_pageviews(name_file, start_time = '20190101', end_time = '20200731'):
    """
    Creates a dataframe from a text file
    param: name_file: name of the text file, start_time: beginning of the desired period, end_time: end of the desired period
    """
    df_topic = create_dataframe(name_file)
    df_topic['Topics'] = remove_space(df_topic)
    df_topic = parse_topics_into_df(df_topic, start_time, end_time)
    df_pageviews = scrape_pageviews(df_topic)
    df_pageviews.fillna(0, inplace=True)
    return df_pageviews, df_topic

def list_valid_topics(df_pageviews):
    lista = df_pageviews.columns.tolist()
    return lista[1:]


### Get translated topics found on Wikidata

In [60]:
def get_label_in_language(english_label, target_language):
    # Endpoint URL for the Wikidata Query Service
    url = "https://query.wikidata.org/sparql"
    
    # SPARQL query to get the item with the English label and its label in the target language
    query = f'''
    SELECT ?item ?itemLabel WHERE {{
      ?item rdfs:label "{english_label}"@en.
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],{target_language}". }}
    }}
    LIMIT 1
    '''
    
    # Headers for the request
    headers = {
        'User-Agent': 'MyBot/0.1 (myemail@example.com)',
        'Accept': 'application/sparql-results+json'
    }
    
    # Make the GET request
    response = requests.get(url, headers=headers, params={'query': query, 'format': 'json'})
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        results = data['results']['bindings']
        if results:
            # Return the item and its label in the target language
            return results[0]['itemLabel']['value']
        else:
            return '_' # No label found for this language
    else:
        # Handle unsuccessful requests
        response.raise_for_status()

def change_Q(name):
    if name.startswith('Q'):
        return ' '
    else:
        return name

def translate_topics(df_topic, lang):
    # Run through all the topics and get the translation in Italien and store it in a new column in the DataFrame
    df_topic['Topics'] = df_topic['Topics'].str.lower()
    df_topic[f'{lang}'] = df_topic['Topics'].apply(lambda x: get_label_in_language(x, lang))
    df_topic[f'{lang}'].replace('No label found for this language', ' ', inplace=True)
    df_topic[f'{lang}'] = df_topic[f'{lang}'].apply(lambda x: change_Q(x))
    return df_topic


### Get pageviews from wikidata for specific country (still not working)

In [114]:
def get_pageviews_for_country(language, project, country, topic, start_date, end_date):

    base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article"
    
    # If a country is specified, include it in the endpoint
    if country:
        endpoint = f"{language}.{project}.org/all-access/all-agents/{topic}/"
    else:
        endpoint = f"{language}.{project}.org/all-access/all-agents"

    # Construct the API URL
    url = f"{base_url}/{endpoint}"

    headers = {
        'User-Agent': 'MyBot/0.1 (myemail@example.com)',
        'Accept': 'application/json'
    }

    # Set parameters
    params = {
        "start": start_date,
        "end": end_date,
        "access": "all-access",
        "agent": "all-agents",
        "granularity": "daily",
        "geotarget": country,
    }

    try:
        # Make the API request
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        # Check if the response is not empty
        if response.text:
            data = response.json()
            df = pd.DataFrame(data["items"])
            return df
        else:
            print("Empty response.")
            return pd.DataFrame()  # Return an empty DataFrame in case of an empty response

    except JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of a JSONDecodeError
    except RequestException as e:
        print(f"Request Exception: {e}")
        print(f"URL: {url}")  # Print the URL for debugging
        return pd.DataFrame()  # Return an empty DataFrame in case of a RequestException

# Example usage
language_code = "en"
project_code = "wikipedia"
country_code = "US"
start_date = "2022010100"
end_date = "2022013100"
topic = "Covid"

result = get_pageviews_for_country(language_code, project_code, country_code, topic, start_date, end_date)

# Output the result
print(result)

Request Exception: 404 Client Error: Not Found for url: https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/Covid/?start=2022010100&end=2022013100&access=all-access&agent=all-agents&granularity=daily&geotarget=US
URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/Covid/
Empty DataFrame
Columns: []
Index: []


### Using google trends API

In [113]:
def get_Trends(topic, country):
    # Create a pytrends object
    pytrends = TrendReq(hl='en-US', tz=10)

    # Build payload with country geotarget
    kw_list = [topic]
    pytrends.build_payload(kw_list, cat=0, timeframe='2019-01-01 2020-07-31', geo=country, gprop='')

    # Get interest over time
    try:
        interest_over_time_df = pytrends.interest_over_time()
        return interest_over_time_df
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

### List all valid topics

In [99]:
lista = list_valid_topics(df_pageviews)
print(lista)
lista = [s.replace('_', ' ').replace("'", "") for s in lista]
print(lista)

# Add to .txt file this list
with open('Self_actualization/self_actualization1.txt', 'w') as f:
    for item in lista:
        f.write("%s\n" % item)

['Abstract_Art', 'Creative_Writing', 'Musical_Composition', 'Graphic_Novels', 'Experimental_Music', 'Abstract_Realism', 'Creative_Industries', 'Creative_Arts', 'Creative_Visualization', 'Creative_Education', 'Creative_Technology', 'Creative_Economy', 'Innovation', 'Imagination', 'Artistry', 'Design', 'Inspiration', 'Expression', 'Ingenuity', 'Originality', 'Inventiveness', 'Creation', 'Vision', 'Talent', 'Craftsmanship', 'Ideation', 'Aesthetics', 'Productivity', 'Composition', 'Insight', 'Motivation', 'Pioneering', 'Resourcefulness', 'Genius', 'Creativity', 'Novelty', 'Mastery', 'Conceptualization', 'Futurism', 'Breakthrough', 'Divergence', 'Artisan', 'Epiphany', 'Experimentation', 'Productiveness', 'Conceptualize', 'Original', 'Renewal', 'Formulation', 'Symbiosis', 'Trailblazing', 'Uniqueness', 'Redefinition', 'Visionary', 'Adaptability', 'Illumination', 'Progression', 'Renaissance', 'Catalyst', 'Transformation', 'Vitality', 'Modernism', 'Contemporary', 'Imaginative', 'Flourishing', '